In [1]:
from eqnemix import eqnemaster
import matplotlib.pyplot as plt

# Main class

### The first thing we need to initialize the class is the location of the station to work with

In [2]:
stax = -103.510147    #Longitude of CI.CLC
stay = 31.408922    # latitude of CI.CLC

eq = eqnemaster(-103.510147,31.408922, deltadist=1000)

### Then we have to define an area around the station to work with

#### Using a Shape file

In [3]:
eq.xminyminshape("extent.shp",inputcrs=4326,outputcrs=3857)

(-11669823.762683755, 3583642.4713136116, 255)

#### By providing the bottom left and the top right corner of the area

In [4]:
import geopandas as gpd
gdf = gpd.read_file("extent.shp") #Leer el archivo
polygon = gdf.geometry.iloc[0]
print(gdf.crs)
print(polygon.bounds)

EPSG:4326
(-104.83181049, 30.6211275, -102.86858939, 32.57773843)


In [5]:
from pyproj import CRS
print(CRS.from_epsg(3587))

EPSG:3587


In [6]:
eq.xminymin(-104.83181049,30.6211275,-102.86858939,32.57773843,inputcrs=4326,outputcrs=3857)

(-11669823.762683755, 3583642.4713136116, 255)

In [7]:
vp_values = [5.5, 5.85, 6.5, 8.18]  
vs_values = [3.06,3.25, 3.61, 4.54]
layer_thickness = [5, 24, 10, 100]  

In [8]:
gridp, grids = eq.nllgrid(vp_values,vs_values,layer_thickness)

In [9]:
tps = eq.eqnefmm()